In [508]:
import pandas as pd
from egbz.utils import *

In [509]:
PATH = '../data/'

# Process des données météos

In [510]:
meteo = pd.read_csv(PATH+'meteo-france.csv', sep=';')
dep = pd.read_csv(PATH+'name.csv')
dep['dep'] = dep.iloc[:, 0].astype('str').apply(lambda x: '0'+x if len(x)==1 else x)
dep = dep.iloc[:, 1:]
meteo = meteo.merge(dep, on='NOM')
meteo.DATE = pd.to_datetime(meteo.DATE, format='%Y%m%d')
meteo.UM = meteo.UM.astype('str')
meteo = meteo[['DATE', 'dep', 'TX', 'UX', 'RR']]

 - *TX*: Température maximale
 - *UX*: Humidité maximale
 - *RR*: Hauteur de précipitation

In [511]:
meteo

,DATE,dep,TX,UX,RR
0,2020-01-01,01,"0,0",98.0,"0,0"
1,2020-01-02,01,"8,7",99.0,"0,0"
2,2020-01-03,01,"9,1",95.0,"0,4"
3,2020-01-04,01,"8,7",97.0,"0,6"
4,2020-01-05,01,"4,9",97.0,"0,0"
...,...,...,...,...,...
53419,2021-04-17,95,"13,5",89.0,"0,0"
53420,2021-04-18,95,"14,3",90.0,"0,0"
53421,2021-04-19,95,"17,1",86.0,"0,0"
53422,2021-04-20,95,"18,2",85.0,"0,0"


# Données de pollution et mobilités

Ces données sont + ou - déjà préprocessées. Juste un détail étrange: il y a des doublons pour le 2021-07-21

In [512]:
pollution_mobility = pd.read_csv(PATH+'pollution_mobility.csv')
pollution_mobility['date'] = pd.to_datetime(pollution_mobility.date)
pollution_mobility['dep'] = (pollution_mobility.numero
                                 .astype('str')
                                 .apply(lambda x: x if len(x)>1 else '0'+x)
                            )
pollution_mobility = pollution_mobility[['date', 'dep', 'pm25', 'pm10', 'mobindex1', 'mobindex2']]
pollution_mobility = pollution_mobility.drop_duplicates()

In [513]:
pollution_mobility

,date,dep,pm25,pm10,mobindex1,mobindex2
0,2020-04-09,03,9.419160,12.640520,-0.56703,0.44839
1,2020-04-10,03,11.835602,15.955548,-0.59055,0.42689
2,2020-04-11,03,13.830579,16.983998,-0.65525,0.46885
3,2020-04-12,03,11.832299,14.632450,-0.68468,0.58674
4,2020-04-13,03,10.151712,11.786599,-0.72576,0.57638
...,...,...,...,...,...,...
47211,2021-08-08,95,4.127157,9.238246,0.04438,0.24186
47212,2021-08-09,95,4.367238,7.922840,-0.03680,0.17273
47213,2021-08-10,95,6.785748,9.272935,-0.04893,0.16384
47214,2021-08-11,95,9.402485,10.441441,-0.05011,0.16131


# Données hospitalières

Il y a pas mal de missing values à prendre en compte. cvdose1 est vide -> on le drop. Puis, regardons le R: il semble être donné de manière irrégulière à partir de juin 2020, on utilise un imputer temporel pour remplir les valeurs manquantes (remarquons qu'avant juillet le R semble être fixé arbitrairement à 1). Pour les taux, il faut également interpoler quelques valeurs et les faires commencer autour de juin

In [561]:
covid_dep = pd.read_csv(PATH+'covid_dep.csv')
covid_fr = pd.read_csv(PATH+'covid_france.csv')

/usr/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [548]:
covid_dep = pd.read_csv(PATH+'covid_dep.csv')
covid_dep['date'] = pd.to_datetime(covid_dep.date)
covid_dep['dep'] = covid_dep['dep'].astype('str')
covid_dep = (
    covid_dep
        .drop(['reg', 'lib_dep', 'lib_reg', 'cv_dose1'], axis=1)
        .set_index('date')
        .interpolate(method='time')
        .dropna()
        .reset_index()
)
covid_dep

/usr/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,date,dep,tx_pos,tx_incid,TO,R,hosp,rea,rad,dchosp,reg_rea,incid_hosp,incid_rea,incid_rad,incid_dchosp,reg_incid_rea,pos,pos_7j
0,2020-06-12,01,1.063201,2.739914,0.080501,1.020000,64,0,409,101,45,0.0,0.0,2.0,1.0,0.0,4.0,18.0
1,2020-06-13,01,0.894988,2.283261,0.073345,1.293333,64,0,409,101,41,0.0,0.0,0.0,0.0,1.0,0.0,15.0
2,2020-06-14,01,0.892857,2.283261,0.073345,1.076667,64,0,409,101,41,0.0,0.0,0.0,0.0,0.0,1.0,15.0
3,2020-06-15,01,0.891266,2.283261,0.069767,0.940000,64,0,410,101,39,0.0,0.0,1.0,0.0,0.0,0.0,15.0
4,2020-06-16,01,0.999412,2.587696,0.067979,0.798000,65,0,410,101,38,2.0,0.0,0.0,0.0,1.0,3.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53338,2021-08-24,976,1.519937,27.552054,0.000000,1.100000,1,0,1275,132,0,0.0,0.0,0.0,0.0,0.0,11.0,77.0
53339,2021-08-25,976,1.545436,26.836416,0.000000,1.100000,1,0,1275,132,0,0.0,0.0,0.0,0.0,0.0,9.0,75.0
53340,2021-08-26,976,1.545436,26.836416,0.000000,1.100000,1,0,1276,132,0,1.0,0.0,1.0,0.0,0.0,9.0,75.0
53341,2021-08-27,976,1.545436,26.836416,0.000000,1.100000,2,0,1276,132,0,1.0,0.0,0.0,0.0,0.0,9.0,75.0


In [568]:
covid_fr = pd.read_csv(PATH+'covid_france.csv')
covid_fr['date'] = pd.to_datetime(covid_fr.date)
covid_fr = (
    covid_fr
        .set_index('date')
        .drop(['cv_dose1', 'esms_cas', 'conf_j1'], axis=1)
        .interpolate(method='time')
        .dropna()
        .reset_index()
)
covid_fr

,date,tx_pos,tx_incid,TO,R,rea,hosp,rad,dchosp,incid_rea,incid_hosp,incid_rad,incid_dchosp,conf,pos,esms_dc,dc_tot,pos_7j
0,2020-06-01,1.496918,5.167251,0.251087,0.830987,1270.0,14237.0,68431.0,18486.0,9.0,89.0,95.0,31.0,152091,171.0,10327.0,28813.0,3468.0
1,2020-06-02,1.508338,5.092752,0.241400,0.826380,1221.0,13978.0,68803.0,18570.0,33.0,229.0,402.0,85.0,151325,723.0,10350.0,28920.0,3418.0
2,2020-06-03,1.488887,4.988453,0.233096,0.826668,1179.0,13465.0,69446.0,18651.0,26.0,250.0,668.0,85.0,151677,659.0,10350.0,29001.0,3348.0
3,2020-06-04,1.500908,4.960143,0.224002,0.831327,1133.0,13054.0,69967.0,18695.0,23.0,195.0,544.0,46.0,152444,622.0,10350.0,29045.0,3329.0
4,2020-06-05,1.504178,4.907994,0.210558,0.839821,1065.0,12650.0,70494.0,18741.0,19.0,213.0,548.0,47.0,153055,560.0,10350.0,29091.0,3294.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,2021-08-24,2.963911,204.192819,0.439106,0.929007,2221.0,11066.0,404141.0,87143.0,214.0,1145.0,871.0,153.0,6649630,21226.0,26505.0,111587.0,137044.0
450,2021-08-25,2.920166,198.121150,0.442665,0.929007,2239.0,11171.0,404823.0,87236.0,182.0,883.0,701.0,93.0,6673336,18814.0,26505.0,111587.0,132969.0
451,2021-08-26,2.920166,198.121150,0.447015,0.929007,2261.0,11104.0,405490.0,87343.0,191.0,741.0,692.0,110.0,6693019,18814.0,26505.0,111587.0,132969.0
452,2021-08-27,2.920166,198.121150,0.448794,0.929007,2270.0,11162.0,406229.0,87438.0,194.0,908.0,750.0,95.0,6711268,18814.0,26505.0,111587.0,132969.0
